Following zhinst blog https://www.zhinst.com/others/en/blogs/hands-superconducting-qubit-characterization and code https://github.com/zhinst/blogs/blob/master/Hands-on%20Superconducting%20Qubit%20Characterization/Single-Qubit-Tuneup.md


In [ ]:
# convenience import for all QCCS software functionality
import numpy as np

import matplotlib.pyplot as plt

from laboneq.simple import *


In [ ]:
my_descriptor=f"""\
instrument_list:
  SHFQC:
  - address: DEV12XXX
    uid: device_shfqc
connections:
  device_shfqc:
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT
    - iq_signal: q0/measure_line
      ports: [QACHANNELS/0/OUTPUT]
    - acquire_signal: q0/acquire_line
      ports: [QACHANNELS/0/INPUT]
"""

# a collection of qubit control and readout parameters as a python dictionary
qubit_parameters = {
    'ro_freq' :  10e6,           # readout frequency of qubit 0 in [Hz] - relative to local oscillator for readout drive upconversion
    'ro_amp' : 0.5,              # readout amplitude
    'ro_amp_spec': 0.05,         # readout amplitude for spectroscopy
    'ro_len' : 1.0e-6,           # readout pulse length in [s]
    'ro_len_spec' : 1.0e-6,      # readout pulse length for resonator spectroscopy in [s]
    'ro_delay': 100e-9,          # readout delay after last drive signal in [s]
    'ro_int_delay' : 180e-9,     # readout line offset calibration - delay between readout pulse and start of signal acquisition in [s]
    
    'qb_freq': 20e6,             # qubit 0 drive frequency in [Hz] - relative to local oscillator for qubit drive upconversion
    'qb_amp_spec': 0.01,         # drive amplitude of qubit spectroscopy
    'qb_len_spec': 15e-6,        # drive pulse length for qubit spectroscopy in [s]
    'qb_len' : 4e-7,             # qubit drive pulse length in [s]
    'pi_amp' : 0.5,              # qubit drive amplitude for pi pulse
    'pi_half_amp' : 0.25,        # qubit drive amplitude for pi/2 pulse
    'qb_t1' : 100e-6,            # qubit T1 time
    'qb_t2' : 100e-6,            # qubit T2 time
    'relax' : 200e-6             # delay time after each measurement for qubit reset in [s]
}

# up / downconversion settings - to convert between IF and RF frequencies
lo_settings = {
    'qb_lo': 4.0e9,              # qubit LO frequency in [Hz]
    'ro_lo': 7.0e9               # readout LO frequency in [Hz]
}

In [ ]:
# function that defines a setup calibration containing the qubit / readout parameters 
def define_calibration(parameters):

     # the calibration object will later be applied to the device setup
    my_calibration = Calibration()

    my_calibration["/logical_signal_groups/q0/drive_line"] = \
        SignalCalibration(
           # each logical signal can have an oscillator associated with it
            oscillator=Oscillator(
                frequency=parameters['qb_freq'],
                modulation_type=ModulationType.HARDWARE
            ),
            local_oscillator=Oscillator(
                frequency=lo_settings['qb_lo'],
            ),
            range=-30
        )
    
    # readout drive line
    my_calibration["/logical_signal_groups/q0/measure_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=parameters['ro_freq'],
                modulation_type=ModulationType.SOFTWARE
            ),
            port_delay=parameters['ro_delay'],
            local_oscillator=Oscillator(
                frequency=lo_settings['ro_lo'],
            ),
           range=-30
       )
     
    # acquisition line
    my_calibration["/logical_signal_groups/q0/acquire_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=parameters['ro_freq'],
                modulation_type=ModulationType.SOFTWARE
            ),
            # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse 
            port_delay=parameters['ro_delay'] + parameters['ro_int_delay'],
            local_oscillator=Oscillator(
                frequency=lo_settings['ro_lo'],
            ),
            range=-30
        )
  
    return my_calibration

In [ ]:
# define the DeviceSetup from descriptor - additionally include information on the dataserver used to connect to the instruments 
my_setup = DeviceSetup.from_descriptor(
    my_descriptor,
    server_host="localhost",
    server_port="8004",
    setup_name="psi",
) 

# define Calibration object based on qubit control and readout parameters
my_calibration = define_calibration(parameters=qubit_parameters)
# apply calibration to device setup
my_setup.set_calibration(my_calibration)

In [ ]:
# perform experiments in emulation mode only? - if True, also generate dummy data for fitting
emulate = True

# create and connect to a session
my_session = Session(device_setup=my_setup)
my_session.connect(do_emulation=emulate)

## Tuneup experiment

In [ ]:
# frequency range of spectroscopy scan - around expected centre frequency as defined in qubit parameters
spec_range = 50e6
# how many frequency points to measure
spec_num = 201

# define sweep parameters for two qubits
freq_sweep_q0 = LinearSweepParameter(uid="res_freq", start=qubit_parameters['ro_freq'] - spec_range / 2, stop=qubit_parameters['ro_freq'] + spec_range / 2, count=spec_num)

# take how many averages per point: 2^n_average
n_average = 14

# spectroscopy excitation pulse
readout_pulse_spec = pulse_library.const(
    length=qubit_parameters['ro_len_spec'], amplitude=qubit_parameters['ro_amp_spec']
)

In [ ]:
# function that defines a resonator spectroscopy experiment, and takes the frequency sweep as a parameter
def res_spectroscopy(freq_sweep):
    # Create resonator spectroscopy experiment - uses only readout drive and signal acquisition
    exp_spec = Experiment(
        uid="Resonator Spectroscopy",
        signals=[
            ExperimentSignal("measure"), 
            ExperimentSignal("acquire"),
        ]
    )         
                
    with exp_spec.acquire_loop_rt(
            uid="shots",
            count=pow(2, n_average),
            acquisition_type=AcquisitionType.SPECTROSCOPY,
            averaging_mode=AveragingMode.SEQUENTIAL
    ):
        with exp_spec.sweep(uid="res_freq", parameter=freq_sweep):
            # readout pulse and data acquisition
            with exp_spec.section(uid="spectroscopy"):
                
                # play resonator excitation pulse
                exp_spec.play(
                    signal="measure", 
                    pulse=readout_pulse_spec
                )
                
                # resonator signal readout 
                exp_spec.acquire(
                    signal="acquire", 
                    handle="res_spec", 
                    length=qubit_parameters['ro_len_spec']
                )
                
            with exp_spec.section(uid="delay"):
                # holdoff time after signal acquisition - minimum 1us required for data processing on UHFQA
                exp_spec.delay(signal="measure", time=1e-6)
    
    return exp_spec

In [ ]:
# function that returns the calibration of the readout line oscillator for the experimental signals
def res_spec_calib(freq_sweep):
        exp_calibration = Calibration()
        # sets the oscillator of the experimental measure signal
        exp_calibration["measure"] = SignalCalibration(
                # for spectroscopy, use the hardware oscillator of the QA, and set the sweep parameter as frequency
                oscillator = Oscillator(
                        "readout_osc",
                        frequency=freq_sweep,
                        modulation_type=ModulationType.HARDWARE
                    ),
                )
                
        return exp_calibration

# signal map - maps the logical signal of the device setup to the experimental signals of the experiment
res_spec_map_q0 = {
    "measure": "/logical_signal_groups/q0/measure_line",
    "acquire": "/logical_signal_groups/q0/acquire_line",
    }

In [ ]:
# define the experiment with the frequency sweep relevant for qubit 0
exp_spec = res_spectroscopy(freq_sweep_q0)

# set signal calibration and signal map for experiment to qubit 0
exp_spec.set_calibration(res_spec_calib(freq_sweep_q0))
exp_spec.set_signal_map(res_spec_map_q0)

# compile the experiment 
compiled_experiment = my_session.compile(exp_spec)

# run the experiment
my_results = my_session.run(compiled_experiment)

In [ ]:
# get the measurement data returned by the instruments from the QCCS session
spec_res = my_results.get_data('res_spec')

# define the frequency axis from the qubit parameters
spec_freq = lo_settings['ro_lo'] + my_results.get_axis('res_spec')[0]

In [ ]:
# plot the measurement data 
fig, ax = plt.subplots(1, 1)
ax.plot(spec_freq / 1e9, abs(spec_res), '.b', label='data')
ax.set_ylabel('Amplitude, $A$ (a.u.)')
ax.set_xlabel('Frequency, $\\nu$ (GHz)')

# increase number of plot points for smooth plotting of fit results
freq_plot = np.linspace(spec_freq[0], spec_freq[-1], 5 * len(spec_freq))

# fit the measurement data to a Fano lineshape to extract the resonance frequency
#popt, pcov = fit_ResSpec(spec_freq, abs(spec_res), 10e6, 7.0e9, 0.5e-6, fano=1, off=1.0e-6, plot=False)
# plot the fit function together with the measurement data
#ax.plot(freq_plot / 1e9, func_Fano(freq_plot, *popt), '-r', label='fit')
ax.legend()

In [ ]:
# choose the readout frequency at the minimum of the fitted lineshape
#qubit_parameters['ro_freq'] = freq_plot[np.argmin(func_Fano(freq_plot, *popt))] - lo_settings['ro_lo']
#print(str((qubit_parameters['ro_freq']+lo_settings['ro_lo'])/1e9) + " GHz")